In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import os
from glob import glob

### 벡터저장소 로드

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

vector_store = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db"
)

print(f"벡터 저장소에 저장된 문서 수: {vector_store._collection.count()}")

벡터 저장소에 저장된 문서 수: 6


In [4]:
# 다중 메시지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=100
)

messages = [
    ("system", "You are a helpful assistant"),
    ("user", "{query}"),
]

prompt = ChatPromptTemplate.from_messages(messages)

print(prompt)

input_variables=['query'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]


In [5]:
print(prompt.input_variables)


['query']


In [6]:
prompt_text = prompt.format_prompt(query="테슬라 창업자는 누구인가요?")
print(prompt_text)

messages=[SystemMessage(content='You are a helpful assistant'), HumanMessage(content='테슬라 창업자는 누구인가요?')]


In [7]:
response = llm.invoke(prompt_text)
print(response.content)


테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다


In [8]:
# LCEL 체인을 구성
chain = prompt | llm
print(chain)

first=ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))]) last=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x000001D87F9E4850>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001D82FD66050>, root_client=<openai.OpenAI object at 0x000001D87E7EDD50>, root_async_client=<openai.AsyncOpenAI object at 0x000001D82FD4EE50>, model_name='gpt-4o-mini', temperature=0.0, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=100)


In [9]:
from pprint import pprint
pprint(chain.input_schema.schema())


{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


In [11]:
response = chain.invoke({"query": "테슬라 창업자는 누구인가요?"})
print(response.content)


테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다


## Prompt + LLM + Output Parser

## StrOutputParser

In [12]:
response

AIMessage(content='테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 26, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'finish_reason': 'length', 'logprobs': None}, id='run-f74c8e20-e383-4786-a9b9-11f408d79498-0', usage_metadata={'input_tokens': 26, 'output_tokens': 100, 'total_tokens': 126})

In [16]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(response)

'테슬라의 창립자는 엘론 머스크(Elon Musk), 마틴 에버하드(Martin Eberhard), 마크 타페닝(Mark Tarpenning), 제프 스프링어(Jeffrey B. Straubel) 등입니다. 테슬라는 2003년에 설립되었으며, 엘론 머스크는 2004년에 투자자로 참여한 후 CEO로 취임하여 회사의 성장에 큰 영향을 미쳤습니다'

In [21]:
str_chain = prompt | llm | output_parser

query = "리비안의 설립 연도는 언제인가요?"
str_response = str_chain.invoke(query)
print(str_response)

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 데 주력하고 있으며, 특히 전기 픽업트럭과 SUV 모델로 주목받고 있습니다.


## Json 출력 - JsonOutputParser

In [23]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()

json_response = chain.invoke("테슬라 창업자는 누구인가요? JSON 형식으로 출력해주세요.")
print(json_response)
print("=========================")

json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "창업자": {\n    "이름": "엘론 머스크",\n    "출생연도": 1971,\n    "국적": "미국",\n    "직업": "기업가, 엔지니어, 발명가"\n  }\n}\n```' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 33, 'total_tokens': 97, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'finish_reason': 'stop', 'logprobs': None} id='run-832fd4f7-0251-4318-b0ec-32549d2366b7-0' usage_metadata={'input_tokens': 33, 'output_tokens': 64, 'total_tokens': 97}
{'창업자': {'이름': '엘론 머스크', '출생연도': 1971, '국적': '미국', '직업': '기업가, 엔지니어, 발명가'}}


In [26]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Information about a person"""

    name: str = Field(description="The name of the person")
    title: str = Field(description="The title or position of the person")

pydantic_parser = PydanticOutputParser(pydantic_object=Person)
# print('Pydantic Output Parser prompt')
# print(pydantic_parser.get_format_instructions())
# print('===============================================')

prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user query. Wrap the output in 'json' tags\n{format_instructions}"),
    ("user", "{query}"),
    
]).partial(format_instructions=pydantic_parser.get_format_instructions()
)


In [30]:
person_chain = prompt | llm | pydantic_parser

response = person_chain.invoke({"query":"테슬라 창업자는 누구인가요?"})
print(response)

name='Elon Musk' title='CEO and Founder of Tesla, Inc.'


### Stream

In [31]:
import time

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    print(chunk.content, end="", flush=True)
    time.sleep(0.1)

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 성장에 큰 영향을 미쳤습니다. 이후 그는 테슬라의 가장 잘 알려진 얼굴이

### Batch

In [33]:
questions = [
    "테슬라 창업자는 누구인가요?",
    "리비안의 설립 연도는 언제인가요?",
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()

================================== Ai Message ==================================

테슬라의 창립자는 엘론 머스크(Elon Musk)입니다. 그러나 테슬라는 2003년에 마틴 에버하드(Martin Eberhard)와 마크 타페닝(Mark Tarpenning)에 의해 설립되었습니다. 엘론 머스크는 2004년에 투자자로 참여한 후, CEO로 취임하며 회사의 방향성을 크게 변화시켰습니다. 이후 그는 테슬라의 가장 유명한 얼굴이
================================== Ai Message ==================================

리비안(Rivian)은 2009년에 설립되었습니다. 이 회사는 전기차를 개발하고 생산하는 데 주력하고 있으며, 특히 전기 픽업트럭과 SUV 모델로 주목받고 있습니다.


### Runnable

In [36]:
# 문서 검색기 생성
retriever = vector_store.as_retriever(
    search_kwargs={"k": 2}
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query)
print(f"검색된 문서 수: {len(retrieved_docs)}")

retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])
pprint(retrieved_docs_text)


검색된 문서 수: 2
('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는\n'
 '이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를')


In [40]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel

runnable = RunnableParallel(
    question=RunnablePassthrough(),
)

runnable.invoke("테슬라 창업자는 누구인가요?")


{'question': '테슬라 창업자는 누구인가요?'}

In [41]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel(
    question=RunnablePassthrough(),
    word_count=RunnableLambda(count_num_words),
)

runnable.invoke("테슬라 창업자는 누구인가요?")


{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

In [42]:
# Prompt 템플릿을 생성
from langchain_core.prompts import ChatPromptTemplate

template = """
Answer the question based only on the following context.
Do not use any external knowledge.
If the answer is not in the context, just say "I don't know".

[Context]
{context}

[Question]
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.pretty_print()



================================ Human Message =================================


Answer the question based only on the following context.
Do not use any external knowledge.
If the answer is not in the context, just say "I don't know".

[Context]
{context}

[Question]
{question}

[Answer]



In [43]:
# 벡터 검색기
retriever = vector_store.as_retriever(
    search_kwargs={"k": 2},
)

# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

# 체인 구성
retriever_chain = retriever | format_docs

# 체인 실행
response = retriever_chain.invoke("테슬라 창업자는 누구인가요?")
pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는\n'
 '\n'
 '이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를')


### RAG Chain 연결

In [44]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=100,
)

# LLM 체인 생성
rag_chain = (
    {"context": retriever_chain, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
response = rag_chain.invoke("테슬라 창업자는 누구인가요?")
print(response)

마틴 에버하드와 마크 타페닝입니다.


In [46]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")
demo.launch()


Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


c:\Users\IloveKOSA\AppData\Local\pypoetry\Cache\virtualenvs\langchain-env-QISotLL5-py3.11\Lib\site-packages\gradio\analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.44.0, however version 4.44.1 is available, please upgrade. 
--------
  warnings.warn(


In [47]:
demo.close()

Closing server running on port: 7861
